imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data"
experimentName="biology_paper_datasizes_BB"

cuda=6

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
    'numOfTrials': 3
}
removes = ['patience','aug_profile']

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix models

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

model /home/elhamod/HGNN/experiments/biology_paper_datasizes_BB/models/4c578e69396e5fdf84b6f8c67ac4e2055811f695d840d2a40ca81f32 -> /home/elhamod/HGNN/experiments/biology_paper_datasizes_BB/models/4c578e69396e5fdf84b6f8c67ac4e2055811f695d840d2a40ca81f32 updated
json /home/elhamod/HGNN/experiments/biology_paper_datasizes_BB/models/4c578e69396e5fdf84b6f8c67ac4e2055811f695d840d2a40ca81f32/params.json updated
model models/831dc79ea7b6e034cacead51aed9d63696e796b862bbf29150dc6e0e could not be fixed to models/831dc79ea7b6e034cacead51aed9d63696e796b862bbf29150dc6e0e
model models/4a786f7dda776afeaa7daeeb82f1330f6962446ac3320b37f2c33c67 could not be fixed to models/4a786f7dda776afeaa7daeeb82f1330f6962446ac3320b37f2c33c67



model models/dcfabf5ce714de42b5bbaeb41b1e0e3730b7e9d950df15420e0548ae could not be fixed to models/dcfabf5ce714de42b5bbaeb41b1e0e3730b7e9d950df15420e0548ae
model models/32879544d09cfb1f7c6b6a8b9cb03b1a8b86a69718b6d152f65341e7 could not be fixed to models/32879544d09cfb1f7c6b6a8b9cb03b1a8b86a69718b6d152f65341e7
model models/814c55ad45d1fb2926f3263807b4412e644ef2c5cb6ee8e41425107e could not be fixed to models/814c55ad45d1fb2926f3263807b4412e644ef2c5cb6ee8e41425107e



model models/25610f5d2832d5369b1c1e631160b92950da7cdbddce764e2c60e3ef could not be fixed to models/25610f5d2832d5369b1c1e631160b92950da7cdbddce764e2c60e3ef
model models/663b0cb31f1785c4e2662da64a5897201cb8d67801369078f636f255 could not be fixed to models/663b0cb31f1785c4e2662da64a5897201cb8d67801369078f636f255
model models/67829df4f9f07949903575bad092abd529fec317b1d4ec9611023eb8 could not be fixed to models/67829df4f9f07949903575bad092abd529fec317b1d4ec9611023eb8




Fix results directory

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result 5e22b0570f3e427d8c05ea1c55d284c0309188738361f92e99ced27a could not be fixed to 5e22b0570f3e427d8c05ea1c55d284c0309188738361f92e99ced27a
result f7ac08342108aa0218195ccec8495f64ff54d6b12a08db45c12028bd could not be fixed to f7ac08342108aa0218195ccec8495f64ff54d6b12a08db45c12028bd
result e278ad9ff3a35ee8f8879b1e050ab79a62473e7213bf4a2474b83bee could not be fixed to e278ad9ff3a35ee8f8879b1e050ab79a62473e7213bf4a2474b83bee



Fix experiments file

In [7]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fix params file

In [8]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        print(experiment_params_new)
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")

{'image_path': 'INHS_cropped', 'suffix': 'biology_paper_cleaned_200max', 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 5e-05, 'numOfTrials': 3, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.6, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool'}
{'image_path': 'INHS_cropped', 'suffix': 'biology_paper_cleaned_100max', 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 5e-05, 'numOfTrials': 3, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.6, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool'}
{'image_path': 'INHS_cropped', 'suffix': 'biology_paper_cleaned_50max', 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 5e-05, 'numOfTrials': 3, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.6, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool'}

json /home/elhamod/HGNN/experiments/biology_paper_datasizes_BB/params.json updated
